# Analyse Milan Flugrouten

## Daten importieren und aufbereiten

In [1]:
# Module importieren
import arcgis
import pandas as pd
import os
import arcpy

In [2]:
# csv-Dateien in Pandas Dataframe importieren
milan1211_csv = r'C:\data\projects\EsriKon2020\Demos\Milane\data\Milan_tag1211_mod.csv'
milan1218_csv = r'C:\data\projects\EsriKon2020\Demos\Milane\data\Milan_tag1218_mod.csv'
milan1211_df = pd.read_csv(milan1211_csv)
milan1218_df = pd.read_csv(milan1218_csv)

# einige Daten anzeigen
milan1218_df.head()

,DeviceID,Year,Month,Day,Hour,Minute,Second,Milisecond,CPUtemperature_C,Pressure_mbar,Temperaturesensor_C,AccelerationX,AccelerationY,AccelerationZ,GyroX,GyroY,GyroZ,MagnetoX,MagnetoY,MagnetoZ,LatitudeN,LongitudeE,Satellites,Speed,Altitude,Battery_voltage,Hdop,Vdop,Searchingtime,Latitude_decimal,Longitude_decimal,Solarvoltage,Csvlinetype,Timestamp
0,1218,2017,7,5,3,0,25,0,24.39,930.8,29.08,0.9883,0.1035,-0.1235,-1.5000,0.1875,-0.5000,38.9648,-16.6992,130.9570,46464895N,007216690E,5.0,0.0,1082.0,4.087,1.4,0.9,52.0,46.774825,7.361150,0.0,1T,2017-07-05T03:00:25.000
1,1218,2017,7,5,3,2,18,0,25.52,936.7,30.34,0.9685,0.1497,-0.1907,-1.3750,-0.1250,-0.6875,38.3789,-14.9414,136.2305,46464985N,007216520E,5.0,0.0,1078.0,4.080,1.4,0.9,34.0,46.774975,7.360867,0.0,1T,2017-07-05T03:02:18.000
2,1218,2017,7,5,3,4,10,0,24.39,934.6,29.88,0.9868,0.1052,-0.1497,-1.8750,-0.0625,-0.5625,38.9648,-17.2852,132.7148,46465581N,007216670E,5.0,0.0,1217.0,4.087,1.4,0.9,24.0,46.775968,7.361117,0.0,1T,2017-07-05T03:04:10.000
3,1218,2017,7,5,3,6,18,0,25.52,933.9,29.80,0.9868,0.1101,-0.1243,-1.6875,0.0000,-0.8750,37.5000,-20.5078,134.7656,46465092N,007216850E,5.0,0.0,1157.0,4.080,1.4,0.9,34.0,46.775153,7.361417,0.0,1T,2017-07-05T03:06:18.000
4,1218,2017,7,5,3,8,46,0,25.52,936.4,30.37,0.9829,0.1174,-0.1396,-1.5000,0.6250,-0.7500,38.6719,-17.5781,137.1094,46465615N,007216630E,7.0,0.0,1099.0,4.080,2.3,0.9,82.0,46.776025,7.361050,0.0,1T,2017-07-05T03:08:46.000


In [3]:
# einige Daten anzeigen
milan1211_df.head()

,DeviceID,Year,Month,Day,Hour,Minute,Second,Milisecond,CPUtemperature_C,Pressure_mbar,Temperaturesensor_C,AccelerationX,AccelerationY,AccelerationZ,GyroX,GyroY,GyroZ,MagnetoX,MagnetoY,MagnetoZ,LatitudeN,LongitudeE,Satellites,Speed,Altitude,Battery_voltage,Hdop,Vdop,Searchingtime,Latitude_decimal,Longitude_decimal,Solarvoltage,Csvlinetype,Timestamp
0,1211,2017,7,2,17,13,58,0,15.35,889.9,21.37,0.3628,-0.9404,-0.4712,4.0625,-7.9375,5.5000,54.4922,117.1875,-11.7188,46447755N,007126010E,7.0,19.5,754.0,3.640,1.2,0.9,22.0,46.746258,7.210017,0.000000,1T,2017-07-02T17:13:58.000
1,1211,2017,7,3,6,25,55,0,13.09,882.2,18.43,0.3694,0.0388,-1.1177,-0.3750,0.1875,-0.7500,-0.2930,154.9805,154.3945,46513387N,007149210E,6.0,0.0,502.0,3.679,1.3,0.9,52.0,46.855645,7.248683,1.241632,1T,2017-07-03T06:25:55.000
2,1211,2017,7,3,6,27,44,0,14.22,885.6,19.39,0.3687,0.0388,-1.1233,-0.3125,0.3125,-0.8750,-2.3438,154.1016,154.1016,46513568N,007149230E,6.0,0.0,596.0,3.663,1.2,0.9,34.0,46.855947,7.248717,1.249344,1T,2017-07-03T06:27:44.000
3,1211,2017,7,3,6,29,43,0,14.22,885.7,19.41,0.3721,0.0427,-1.1147,-0.3125,0.2500,-0.9375,0.0000,152.3438,152.3438,46513977N,007149120E,6.0,0.0,846.0,3.663,1.3,0.9,34.0,46.856628,7.248533,1.249344,1T,2017-07-03T06:29:43.000
4,1211,2017,7,3,6,31,43,0,14.22,886.5,19.59,0.3691,0.0356,-1.1169,-0.1250,0.1875,-0.6875,-0.8789,154.3945,155.5664,46514155N,007149110E,6.0,1.0,987.0,3.663,1.3,0.9,34.0,46.856925,7.248517,1.257056,1T,2017-07-03T06:31:43.000


In [4]:
# Überprüfen, ob es Datensätze mit Null-Werten gibt in den Attributen Lat./Long. und Timestamp
df = milan1211_df # jeweils anpassen
attr = 'Latitude_decimal' # (Latitude_decimal, Longitude_decimal, Timestamp)jeweils anpassen um zu überprüfen, ob ein Attribut Null-Werte beinhaltet
df.loc[df[attr].isnull()]

,DeviceID,Year,Month,Day,Hour,Minute,Second,Milisecond,CPUtemperature_C,Pressure_mbar,Temperaturesensor_C,AccelerationX,AccelerationY,AccelerationZ,GyroX,GyroY,GyroZ,MagnetoX,MagnetoY,MagnetoZ,LatitudeN,LongitudeE,Satellites,Speed,Altitude,Battery_voltage,Hdop,Vdop,Searchingtime,Latitude_decimal,Longitude_decimal,Solarvoltage,Csvlinetype,Timestamp
4842,1211,2017,7,12,12,1,22,0,25.52,924.3,30.92,0.0972,-0.1731,-0.1741,-115.8125,-6.5625,51.6250,-34.2773,113.3789,133.3008,[timeout],[timeout0],NaN,NaN,NaN,4.172,NaN,NaN,NaN,NaN,NaN,1.665792,1T,2017-07-12T12:01:22.000
5290,1211,2017,7,13,8,57,22,0,22.13,918.7,28.55,-0.0308,0.1663,-1.0706,9.8750,-5.0000,-3.8750,25.4883,58.3008,143.2617,[timeout],[timeout0],NaN,NaN,NaN,4.157,NaN,NaN,NaN,NaN,NaN,1.303328,1T,2017-07-13T08:57:22.000
8858,1211,2017,7,19,19,53,22,0,26.65,932.4,33.18,0.9631,0.2019,-0.4893,11.5000,-2.3125,-1.0625,-40.4297,73.8281,128.9062,[timeout],[timeout0],NaN,NaN,NaN,4.087,NaN,NaN,NaN,NaN,NaN,0.000000,1T,2017-07-19T19:53:22.000
10308,1211,2017,7,22,14,13,22,0,28.91,937.2,34.58,0.3970,-0.1672,-0.4744,338.7500,-57.2500,-40.4375,22.2656,150.0000,121.2891,[timeout],[timeout0],NaN,NaN,NaN,4.172,NaN,NaN,NaN,NaN,NaN,0.647808,1T,2017-07-22T14:13:22.000
17742,1211,2017,8,5,10,1,22,0,28.91,942.9,35.10,0.6096,0.0945,0.4790,51.5625,1.3750,-36.0000,-0.2930,92.2852,152.0508,[timeout],[timeout0],NaN,NaN,NaN,4.103,NaN,NaN,NaN,NaN,NaN,1.449856,1T,2017-08-05T10:01:22.000
22119,1211,2017,8,14,10,13,22,0,23.26,921.7,28.57,-0.0066,0.1689,-1.2683,-8.0000,25.3750,-25.5625,26.3672,84.9609,150.5859,[timeout],[timeout0],NaN,NaN,NaN,4.064,NaN,NaN,NaN,NaN,NaN,1.218496,1T,2017-08-14T10:13:22.000
22120,1211,2017,8,14,10,15,22,0,23.26,923.8,29.19,0.0171,-0.4268,-0.9949,53.0000,-135.4375,2.0000,29.0039,44.8242,133.3008,[timeout],[timeout0],NaN,NaN,NaN,4.064,NaN,NaN,NaN,NaN,NaN,1.303328,1T,2017-08-14T10:15:22.000
30973,1211,2017,8,30,17,21,22,0,26.65,939.0,33.25,1.0115,0.1628,-0.5103,0.5625,-0.7500,-2.6875,-39.8438,67.3828,127.7344,[timeout],[timeout0],NaN,NaN,NaN,4.072,NaN,NaN,NaN,NaN,NaN,0.000000,1T,2017-08-30T17:21:22.000
36512,1211,2017,9,10,6,11,22,0,19.87,912.7,25.29,1.0503,0.0522,-0.4543,1.2500,-0.7500,-1.1875,-60.9375,124.8047,79.6875,[timeout],[timeout0],NaN,NaN,NaN,3.733,NaN,NaN,NaN,NaN,NaN,1.388160,1T,2017-09-10T06:11:22.000
40836,1211,2017,9,19,6,7,22,0,17.61,908.3,23.47,1.0535,0.1194,-0.4138,1.0000,0.5000,-0.9375,-66.2109,72.0703,56.2500,[timeout],[timeout0],NaN,NaN,NaN,3.725,NaN,NaN,NaN,NaN,NaN,0.902304,1T,2017-09-19T06:07:22.000


In [5]:
# Detailliertere Angaben zu den Attributen mit Null-Werten
null_values_count = df.loc[df[attr].isnull()].shape[0]
print('Attribut: ' + attr)
print('Datensätze mit Null-Werten: ' + str(null_values_count))
print('Total: '+ str(len(df)))
null_values_perc = null_values_count*100/len(df)
print('prozentualer Anteil: ' + str(null_values_perc))

Attribut: Latitude_decimal
Datensätze mit Null-Werten: 11
Total: 42955
prozentualer Anteil: 0.02560819462227913


In [6]:
# Attribute mit Null-Werten aus dem DataFrame entfernen
# so neu definierter DataFrame hat keine rel. Attribute mit Null-Werten mehr
milan1211_df = milan1211_df.loc[milan1211_df['Latitude_decimal'].notnull()].copy()
milan1218_df = milan1218_df.loc[milan1218_df['Latitude_decimal'].notnull()].copy()

In [7]:
# Nur diejenigen GPS-Punkte verwenden, welche 'Speed nicht gleich 0.0 haben'? > nein, verfälscht das Bild
# milan1218_df = milan1211_df.loc[milan1211_df['Speed'] != 0.0].copy()
# print('Total: '+ str(len(milan1218_df)))

Total: 8249


In [8]:
# Georeferenzierung (neuer DataFrame hat anschliessend Shape-Attribut)
milan1211_df = arcgis.features.GeoAccessor.from_xy(milan1211_df,'Longitude_decimal','Latitude_decimal',sr=4326)
milan1218_df = arcgis.features.GeoAccessor.from_xy(milan1218_df,'Longitude_decimal','Latitude_decimal',sr=4326)
milan1211_df

,DeviceID,Year,Month,Day,Hour,Minute,Second,Milisecond,CPUtemperature_C,Pressure_mbar,Temperaturesensor_C,AccelerationX,AccelerationY,AccelerationZ,GyroX,GyroY,GyroZ,MagnetoX,MagnetoY,MagnetoZ,LatitudeN,LongitudeE,Satellites,Speed,Altitude,Battery_voltage,Hdop,Vdop,Searchingtime,Latitude_decimal,Longitude_decimal,Solarvoltage,Csvlinetype,Timestamp,SHAPE
0,1211,2017,7,2,17,13,58,0,15.35,889.9,21.37,0.3628,-0.9404,-0.4712,4.0625,-7.9375,5.5000,54.4922,117.1875,-11.7188,46447755N,007126010E,7.0,19.5,754.0,3.640,1.2,0.9,22.0,46.746258,7.210017,0.000000,1T,2017-07-02T17:13:58.000,"{'x': 7.2100167, 'y': 46.7462583, 'spatialRefe..."
1,1211,2017,7,3,6,25,55,0,13.09,882.2,18.43,0.3694,0.0388,-1.1177,-0.3750,0.1875,-0.7500,-0.2930,154.9805,154.3945,46513387N,007149210E,6.0,0.0,502.0,3.679,1.3,0.9,52.0,46.855645,7.248683,1.241632,1T,2017-07-03T06:25:55.000,"{'x': 7.2486833, 'y': 46.855645, 'spatialRefer..."
2,1211,2017,7,3,6,27,44,0,14.22,885.6,19.39,0.3687,0.0388,-1.1233,-0.3125,0.3125,-0.8750,-2.3438,154.1016,154.1016,46513568N,007149230E,6.0,0.0,596.0,3.663,1.2,0.9,34.0,46.855947,7.248717,1.249344,1T,2017-07-03T06:27:44.000,"{'x': 7.2487167, 'y': 46.8559467, 'spatialRefe..."
3,1211,2017,7,3,6,29,43,0,14.22,885.7,19.41,0.3721,0.0427,-1.1147,-0.3125,0.2500,-0.9375,0.0000,152.3438,152.3438,46513977N,007149120E,6.0,0.0,846.0,3.663,1.3,0.9,34.0,46.856628,7.248533,1.249344,1T,2017-07-03T06:29:43.000,"{'x': 7.2485333, 'y': 46.8566283, 'spatialRefe..."
4,1211,2017,7,3,6,31,43,0,14.22,886.5,19.59,0.3691,0.0356,-1.1169,-0.1250,0.1875,-0.6875,-0.8789,154.3945,155.5664,46514155N,007149110E,6.0,1.0,987.0,3.663,1.3,0.9,34.0,46.856925,7.248517,1.257056,1T,2017-07-03T06:31:43.000,"{'x': 7.2485167, 'y': 46.856925, 'spatialRefer..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42950,1211,2017,9,23,4,34,5,0,25.52,939.4,32.49,1.0081,-0.0857,-0.5203,14.5625,23.3125,-1.7500,-67.6758,85.2539,65.9180,46448894N,007109300E,6.0,0.5,774.0,3.995,1.6,0.9,14.0,46.748157,7.182167,0.000000,1T,2017-09-23T04:34:05.000,"{'x': 7.1821667, 'y': 46.7481567, 'spatialRefe..."
42951,1211,2017,9,23,4,36,10,0,25.52,934.9,31.24,1.1436,0.1309,-0.5242,-6.6250,4.1875,-5.8750,-66.2109,72.6562,69.7266,46448926N,007109330E,9.0,0.0,774.0,3.995,1.0,0.7,22.0,46.748210,7.182217,0.000000,1T,2017-09-23T04:36:10.000,"{'x': 7.182216700000001, 'y': 46.74821, 'spati..."
42952,1211,2017,9,23,4,38,6,0,24.39,933.2,30.64,0.9924,-0.2590,-0.4712,-0.5625,0.1875,-1.2500,-68.2617,87.5977,68.2617,46448942N,007109340E,9.0,0.0,774.0,3.995,1.2,0.8,12.0,46.748237,7.182233,0.000000,1T,2017-09-23T04:38:06.000,"{'x': 7.1822333, 'y': 46.7482367, 'spatialRefe..."
42953,1211,2017,9,23,4,41,38,0,30.04,946.3,35.19,0.9902,-0.2593,-0.4495,-0.3750,-0.1875,-0.8750,-64.1602,84.6680,66.5039,46448874N,007109380E,7.0,0.0,744.0,3.956,1.5,0.8,14.0,46.748123,7.182300,0.000000,1T,2017-09-23T04:41:38.000,"{'x': 7.1823, 'y': 46.748123299999996, 'spatia..."


In [9]:
# Spatial DataFrame in eine FileGeodatabase schreiben
fgdb = r'Milane_Datenimport.gdb' # jeweils anpassen
fc_milan_name = 'Milan1211' # jeweils anpassen
df = milan1211_df # jeweils anpassen
fc_milan = df.spatial.to_featureclass(os.path.join(fgdb,fc_milan_name))
fc_milan

'C:\\data\\projects\\EsriKon2020\\Demos\\Milane\\Milane_Datenimport.gdb\\Milan1211'